FILE NEEDED :

- video demo
- dataset.csv
- SW_tambahan.txt
- normal_list.txt
- Buat Folder "wav"
---



Penginputan Video Yang Mau Dites

In [ ]:
video = input("nama video yang akan dites: ")
video = video+".mp4"

nama video yang akan dites: videoplayback


Install Library

In [ ]:
!pip install SpeechRecognition
!pip install sastrawi
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Import Library & Setting

In [ ]:
#Library Baca Data
import pandas as pd

#Library Video Menjadi Text
import moviepy.editor as mp
import speech_recognition as sr

#Library Seleksi Data
#Library Preprocessing
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#Library Train Data
from sklearn.ensemble import RandomForestClassifier

#--------------------------------------------------
additional_stopwords = "SW_tambahan.txt"
normalization_list = "normal_list.txt"

#NORMLIZER
normal_d = {}
with open(normalization_list,'r') as f:
  for line in f:
    line = line.strip('\n')
    line = line.split(' ')
    kunci = line[0]
    nilai = line[1]
    normal_d[kunci] = nilai

#STEMMER
stemfactory = StemmerFactory()
stemmer = stemfactory.create_stemmer()

#STOPWORDS SETTING
indo_SW_NLTK = set(stopwords.words('indonesian'))
stopword_tambahan = []
with open(additional_stopwords,'r') as f:
  for line in f:
    line = line.strip()
    stopword_tambahan.append(line) 
for i in indo_SW_NLTK:
  kata = i
  stopword_tambahan.append(i)
stopword_all = StopWordRemoverFactory().get_stop_words() + stopword_tambahan

# **MEMBERSIHKAN DATA DEMO**

---
## Video Dijadiin Text
---



In [ ]:
title = video[:-4]
print("MP4 to WAV preparation for ",video)
tampung = " "
clip = mp.AudioFileClip(video)

clip_duration = int(clip.duration)
total = int(int(clip.duration)/120)+1
dur = int(clip_duration/total)

chunk = 1
akhir = 0
tampung = ""

print("durasi = ",clip_duration)
print("total chunk : ",total)
print("durasi chunk : ",dur)
print()
print("chunk processing...")

#MAKING WAV'S CHUNKS FROM MP4
while(chunk<=total):
  awal = akhir
  akhir = dur*chunk
  if (chunk==1):
    print("chunk [1]")
    clips = clip.subclip(0,akhir)
  elif (chunk==total):
    print("chunk [last]")
    clips = clip.subclip(awal)
  else:
    print("chunk [",str(chunk),"]")
    clips = clip.subclip(awal,akhir)
  print()
  print("making chunk number,",chunk,"...")
  clips.write_audiofile(("wav//"+title+"("+str(chunk)+")"+".wav"))

  #CONVERTING WAV'S CHUNK TO TEXT
  r = sr.Recognizer()
  with sr.AudioFile("wav//"+title+"("+str(chunk)+")"+".wav") as sumber:
    audio_text = r.listen(sumber)
    try:
      print('CONVERTING....')
      text = r.recognize_google(audio_text,language="id-ID")
      tampung = tampung+' '+text
    except:
      print('Error converting : ',title)
  chunk = chunk+1
print("CONVERTED.")
#INPUT TEXT TO DICTIONARY
data = {'file_name':[],'text':[]}
data['file_name'].append(title)
data['text'].append(tampung)

MP4 to WAV preparation for  videoplayback.mp4
durasi =  421
total chunk :  4
durasi chunk :  105

chunk processing...
chunk [1]

making chunk number, 1 ...
[MoviePy] Writing audio in wav//videoplayback(1).wav


100%|██████████| 2316/2316 [00:01<00:00, 2314.02it/s]

[MoviePy] Done.


CONVERTING....
chunk [ 2 ]

making chunk number, 2 ...
[MoviePy] Writing audio in wav//videoplayback(2).wav


100%|██████████| 2316/2316 [00:01<00:00, 2276.08it/s]

...

[MoviePy] Done.


CONVERTING....
chunk [last]

making chunk number, 4 ...
[MoviePy] Writing audio in wav//videoplayback(4).wav


100%|██████████| 2355/2355 [00:01<00:00, 2330.99it/s]

[MoviePy] Done.


CONVERTING....
CONVERTED.


---
## Preprocessing Text Demo
--- 

In [ ]:
data['cleared_text']=" "
teks = data['text'][0]
print("Mengecilkan huruf...")
teks = teks.lower()
print("Mengilangkan angka...")
teks = re.sub(r"\d+", "", teks)
print("Tokenizing...")
teks = teks.split()

print("Menormalisasi kata...")
#NORMALISASI
tmp =[]
for j in teks:
  try: a = normal_d[j]
  except : a=j
  finally: tmp.append(a)
teks = tmp

print("Menghapus Stopwords...")
#STOPWORDS
teks = [word for word in teks if not word in stopword_all]

print("Memulai Stemming...")
#STEMMING
teks = ' '.join(teks)
teks = stemmer.stem(teks)

print("Menyimpan...")
data['cleared_text']=teks

Mengecilkan huruf...
Mengilangkan angka...
Tokenizing...
Menormalisasi kata...
Menghapus Stopwords...
Memulai Stemming...
Menyimpan...


#KLASIFIKASI DATA DEMO

In [ ]:
data_train = pd.read_csv("dataset.csv")

data_test = pd.DataFrame(data)
data_test = data_test.drop(['text','file_name'],axis=1)

#MEMBUAT MODEL BAG OF WORDS
cv = CountVectorizer()
X_train = cv.fit_transform(data_train['clear_text']).toarray()
Y_train = data_train.iloc[:,1].values
X_test = cv.transform(data_test['cleared_text']).toarray()

# Metode Random Forest
crf = RandomForestClassifier(n_estimators = 14, random_state = 0)
crf.fit(X_train, Y_train)

Y_pred = crf.predict(X_test)

if Y_pred==1:
  rate = 5
  arti = "Video berisi ulasan positif yang menyatakan pengguna menyukai keseluruhan produk."
elif Y_pred==2:
  rate = 4
  arti = "Video berisi ulasan agak positif yang menyatakan pengguna menyukai produk namun terdapat sedikit hal yang kurang disukai dari produk."
elif Y_pred==3:
  rate = 3
  arti = "Video berisi ulasan netral yang menyatakan tingkat kesukaan dan ketidaksukaan terhadap produk seimbang."
elif Y_pred==4:
  rate = 2
  arti = "Video berisi ulasan agak negatif yang menyatakan pengguna kurang menyukai produk namun masih ada hal yang disukai dari produk."
else :
  rate = 1
  arti = "Video berisi ulasan negatif yang menyatakan pengguna tidak menyukai keseluruhan produk."

print("rating ulasan produk pada",video,"adalah",rate)
print(arti)

rating ulasan produk pada videoplayback.mp4 adalah 4
Video berisi ulasan agak positif yang menyatakan pengguna menyukai produk namun terdapat sedikit hal yang kurang disukai dari produk.


In [ ]:
'''
import shutil
shutil.rmtree("wav")
'''